<a href="https://colab.research.google.com/github/rhilderbrand/MSDS-422-MachineLearning/blob/main/Hilderbrand_Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 6: Neural Networks

This assignment asks you to fit a number of neural networks, comparing processing time and performance across experimental treatments. Processing time will be recorded for the fitting on the train.csv.  Kaggle.com accuracy scores will be reported for all benchmarks

## System Setup & Data Preparation

In [1]:
# prepare for Python version 3 features and functions
from __future__ import division, print_function, unicode_literals

# import common packages
import pandas as pd
import numpy as np
import os

# import TensorFlow
import tensorflow as tf

#suppress tf.logging
import logging
logging.getLogger('tensorflow').disabled = True

# for visualization
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#S2 Mount Google Drive to Colab Enviorment
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#S3 Establish working directory
import os # Operation System
os.getcwd()
%cd /content/gdrive/My Drive/Colab Notebooks/Week 6/
!pwd
!ls
print('Working Directory')
print(os.getcwd())

work_dir = " /content/gdrive/My Drive/Colab Notebooks/Week 6/"

/content/gdrive/My Drive/Colab Notebooks/Week 6
/content/gdrive/My Drive/Colab Notebooks/Week 6
Hilderbrand_Assignment_6  test.csv  train.csv
Working Directory
/content/gdrive/My Drive/Colab Notebooks/Week 6


In [4]:
# split data into train and test sets
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [5]:
# check data splits for test and train
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 784)
(10000, 784)
(60000,)
(10000,)


##Model 1 - 2 layers 10 nodes per layer

**Processing Time:** 128 s

**Training Set Accuracy:** 0.8613

**Test Set Accuracy:** 0.8637

In [6]:
%%time
feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf_1 = tf.estimator.DNNClassifier(hidden_units=[10,10], n_classes=10,
                                     feature_columns=feature_cols)

input_fn_1 = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=50, batch_size=50, shuffle=True)
dnn_clf_1.train(input_fn=input_fn_1)

CPU times: user 2min 13s, sys: 7.5 s, total: 2min 21s
Wall time: 1min 32s


In [7]:
# evaluate training accuracy
train_1_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, shuffle=False)
eval_results_train_1 = dnn_clf_1.evaluate(input_fn=train_1_input_fn)
eval_results_train_1

{'accuracy': 0.87925,
 'average_loss': 0.45133743,
 'global_step': 60000,
 'loss': 0.45131588}

In [8]:
# evalute test accuracy
test_1_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results_test_1 = dnn_clf_1.evaluate(input_fn=test_1_input_fn)
eval_results_test_1

{'accuracy': 0.8842,
 'average_loss': 0.4361465,
 'global_step': 60000,
 'loss': 0.43457457}

In [9]:
#confusion matrix for Model 1 predictions
raw_predictions_1 = dnn_clf_1.predict(input_fn=test_1_input_fn)
predictions_1 = [p['class_ids'][0] for p in raw_predictions_1]
confusion_matrix_1 = tf.math.confusion_matrix(y_test, predictions_1)
confusion_matrix_1

<tf.Tensor: shape=(10, 10), dtype=int32, numpy=
array([[ 939,    0,    3,    1,    0,   14,   12,    3,    8,    0],
       [   0, 1090,    3,    4,    0,    3,    5,    2,   28,    0],
       [  13,   16,  900,   16,    5,    1,   23,   17,   34,    7],
       [   3,    2,   32,  888,    0,   22,    5,   12,   36,   10],
       [   6,    1,    1,    2,  836,    5,   27,    2,   14,   88],
       [  15,    4,   10,   78,   24,  682,   17,   15,   37,   10],
       [  21,    4,   10,    0,    3,   27,  889,    2,    2,    0],
       [   7,   19,   27,    2,   20,    0,    4,  899,    8,   42],
       [  10,   11,   13,   18,    6,   45,   15,   15,  827,   14],
       [  16,    3,    3,   11,   45,    4,    0,   21,   14,  892]],
      dtype=int32)>

In [10]:
print(predictions_1)

[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 2, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 5, 7, 2, 7, 1, 2, 1, 1, 7, 9, 2, 3, 5, 3, 2, 4, 4, 6, 3, 5, 5, 2, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 2, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 7, 9, 7, 9, 6, 2, 7, 8, 4, 7, 5, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 1, 3, 9, 7, 9, 4, 4, 9, 8, 5, 9, 7, 6, 9, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 9, 6, 7, 3, 1, 7, 1, 8, 2, 0, 8, 9, 9, 5, 8, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 3, 0, 1, 1, 1, 0, 4, 0, 8, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 7, 9, 3, 9, 0, 3, 5, 7, 3, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 7, 7, 7, 2, 2, 4, 1, 5, 8, 8, 4, 2, 6, 0, 6, 4, 2, 4, 1, 9, 5, 7, 7, 2, 1, 2, 0, 8, 3, 7, 7, 8, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2, 6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 6, 7, 1, 2, 9, 2, 2, 9, 4, 3, 3, 0, 0, 5, 1, 9, 6, 5, 3, 5, 1, 7, 9, 3, 0, 9, 2, 0, 7, 1, 1, 2, 1, 

In [12]:
kag_df1 = pd.DataFrame(predictions_1, columns= ['Label']).reset_index()
kag_df1['index'] = kag_df1.index + 1
kag_df1.rename(columns ={'index':'ImageId'}, inplace=True)
kag_df1.head()

,ImageId,Label
0,1,7
1,2,2
2,3,1
3,4,0
4,5,4


In [13]:
kag_df1.to_csv('Hilderbrand_DNN_Predictions_1.csv')

##Model 2 - 2 layers 20 nodes per layer

**Processing Time:** 140 s

**Training Set Accuracy:** 0.90785

**Test Set Accuracy:** 0.9142

In [14]:
%%time
feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf_2 = tf.estimator.DNNClassifier(hidden_units=[20,20], n_classes=10,
                                     feature_columns=feature_cols)

input_fn_2 = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=50, batch_size=50, shuffle=True)
dnn_clf_2.train(input_fn=input_fn_2)

CPU times: user 2min 31s, sys: 8.57 s, total: 2min 40s
Wall time: 1min 43s


In [15]:
# evaluate training accuracy
train_2_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, shuffle=False)
eval_results_train_2 = dnn_clf_2.evaluate(input_fn=train_2_input_fn)
eval_results_train_2

{'accuracy': 0.9058167,
 'average_loss': 0.33876786,
 'global_step': 60000,
 'loss': 0.33879754}

In [16]:
# evalute test accuracy
test_2_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results_test_2 = dnn_clf_2.evaluate(input_fn=test_2_input_fn)
eval_results_test_2

{'accuracy': 0.9113,
 'average_loss': 0.32780066,
 'global_step': 60000,
 'loss': 0.3263556}

In [17]:
#confusion matrix for Model 2 predictions
raw_predictions_2 = dnn_clf_2.predict(input_fn=test_2_input_fn)
predictions_2 = [p['class_ids'][0] for p in raw_predictions_2]
confusion_matrix_2 = tf.math.confusion_matrix(y_test, predictions_2)
confusion_matrix_2

<tf.Tensor: shape=(10, 10), dtype=int32, numpy=
array([[ 956,    0,    4,    1,    0,    6,    7,    1,    5,    0],
       [   0, 1106,    4,    3,    1,    0,    3,    2,   16,    0],
       [  11,    9,  902,   15,   13,    2,   20,   11,   41,    8],
       [   1,    1,   25,  903,    0,   29,    2,   14,   27,    8],
       [   2,    5,    5,    1,  912,    0,   13,    2,    7,   35],
       [  11,    1,    7,   47,   12,  740,   15,   15,   37,    7],
       [  16,    3,    6,    0,   11,   16,  899,    2,    4,    1],
       [   4,   21,   20,    5,   11,    0,    0,  936,    3,   28],
       [   8,    7,    9,   23,    7,   26,   11,   14,  860,    9],
       [  13,    9,    3,   14,   39,    7,    0,   20,    5,  899]],
      dtype=int32)>

In [18]:
print(predictions_2)

[7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 4, 2, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 1, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 4, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 9, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 4, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 3, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 7, 9, 3, 9, 0, 3, 5, 7, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 7, 7, 9, 2, 2, 4, 1, 5, 8, 8, 7, 2, 5, 0, 2, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 0, 8, 5, 7, 7, 9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2, 6, 4, 1, 5, 4, 2, 9, 2, 0, 4, 0, 0, 2, 8, 6, 7, 1, 2, 4, 5, 2, 9, 4, 3, 3, 0, 0, 5, 1, 9, 6, 5, 3, 5, 7, 7, 9, 3, 0, 9, 2, 0, 7, 1, 1, 2, 1, 

In [19]:
kag_df2 = pd.DataFrame(predictions_2, columns= ['Label']).reset_index()
kag_df2['index'] = kag_df2.index + 1
kag_df2.rename(columns ={'index':'ImageId'}, inplace=True)
kag_df2.head()

,ImageId,Label
0,1,7
1,2,2
2,3,1
3,4,0
4,5,4


In [20]:
kag_df2.to_csv('Hilderbrand_DNN_Predictions_2.csv')

##Model 3 - 5 layers 10 nodes per layer

**Processing Time:** 138 s

**Training Set Accuracy:** 0.7168

**Test Set Accuracy:** 0.718

In [21]:
%%time
feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf_3 = tf.estimator.DNNClassifier(hidden_units=[10,10,10,10,10], n_classes=10,
                                     feature_columns=feature_cols)

input_fn_3 = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=50, batch_size=50, shuffle=True)
dnn_clf_3.train(input_fn=input_fn_3)

CPU times: user 2min 26s, sys: 7.29 s, total: 2min 33s
Wall time: 1min 37s


In [22]:
# evaluate training accuracy
train_3_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, shuffle=False)
eval_results_train_3 = dnn_clf_3.evaluate(input_fn=train_3_input_fn)
eval_results_train_3

{'accuracy': 0.69053334,
 'average_loss': 1.0485452,
 'global_step': 60000,
 'loss': 1.0485368}

In [23]:
# evalute test accuracy
test_3_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results_test_3 = dnn_clf_3.evaluate(input_fn=test_3_input_fn)
eval_results_test_3

{'accuracy': 0.6993,
 'average_loss': 1.0158482,
 'global_step': 60000,
 'loss': 1.0182697}

In [24]:
#confusion matrix for Model 3 predictions
raw_predictions_3 = dnn_clf_3.predict(input_fn=test_3_input_fn)
predictions_3 = [p['class_ids'][0] for p in raw_predictions_3]
confusion_matrix_3 = tf.math.confusion_matrix(y_test, predictions_3)
confusion_matrix_3

<tf.Tensor: shape=(10, 10), dtype=int32, numpy=
array([[ 744,    0,    4,    2,    0,  211,   10,    2,    7,    0],
       [   0, 1090,    9,    0,    9,    2,    9,    2,   14,    0],
       [   7,   35,  439,    5,    3,   43,  405,    5,   89,    1],
       [  24,   11,    2,  503,   19,  309,   16,   60,   47,   19],
       [   0,   29,    1,    3,  798,    2,   12,    0,   38,   99],
       [  11,    7,    3,  284,    3,  435,   37,   32,   68,   12],
       [  13,   20,  163,    0,    2,   45,  683,    0,   32,    0],
       [   0,   17,    0,   47,   15,    7,    8,  792,   58,   84],
       [   0,   23,    7,   20,   12,   41,   39,   79,  744,    9],
       [   1,    4,    3,   28,  137,   12,    0,   38,   21,  765]],
      dtype=int32)>

In [25]:
print(predictions_3)

[7, 2, 1, 0, 4, 1, 9, 4, 1, 9, 0, 6, 9, 0, 1, 5, 9, 3, 8, 4, 9, 6, 1, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 5, 7, 2, 7, 1, 6, 1, 1, 7, 4, 2, 3, 5, 4, 6, 4, 4, 6, 3, 3, 3, 2, 5, 4, 1, 9, 5, 7, 6, 9, 8, 7, 9, 6, 4, 3, 0, 7, 0, 5, 9, 1, 9, 3, 8, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 4, 9, 2, 9, 6, 0, 3, 4, 9, 9, 5, 1, 9, 4, 8, 8, 5, 9, 8, 4, 4, 4, 9, 8, 3, 1, 7, 6, 9, 4, 0, 3, 8, 3, 6, 6, 5, 8, 8, 1, 0, 1, 6, 4, 6, 7, 7, 1, 9, 1, 8, 2, 0, 1, 4, 9, 5, 5, 1, 3, 6, 0, 0, 4, 4, 2, 5, 4, 8, 5, 4, 3, 1, 4, 4, 8, 5, 5, 6, 3, 1, 8, 1, 8, 1, 8, 8, 0, 8, 4, 2, 5, 0, 1, 1, 4, 0, 9, 5, 3, 1, 6, 9, 6, 5, 6, 1, 1, 1, 3, 9, 5, 2, 9, 9, 3, 9, 4, 9, 5, 3, 5, 3, 5, 7, 6, 6, 7, 1, 6, 8, 4, 1, 7, 5, 3, 6, 7, 7, 9, 6, 2, 4, 1, 5, 7, 8, 9, 6, 5, 5, 1, 4, 2, 4, 1, 9, 5, 7, 7, 2, 6, 2, 0, 8, 3, 7, 7, 9, 1, 8, 1, 5, 0, 3, 0, 1, 9, 9, 4, 1, 8, 6, 1, 2, 4, 8, 5, 9, 2, 6, 4, 1, 8, 4, 2, 9, 2, 0, 4, 0, 0, 2, 8, 1, 8, 1, 6, 4, 0, 2, 9, 4, 5, 3, 0, 0, 3, 1, 9, 6, 9, 5, 5, 7, 8, 9, 3, 5, 8, 2, 0, 8, 1, 1, 2, 1, 

In [26]:
kag_df3 = pd.DataFrame(predictions_3, columns= ['Label']).reset_index()
kag_df3['index'] = kag_df3.index + 1
kag_df3.rename(columns ={'index':'ImageId'}, inplace=True)
kag_df3.head()

,ImageId,Label
0,1,7
1,2,2
2,3,1
3,4,0
4,5,4


In [27]:
kag_df3.to_csv('Hilderbrand_DNN_Predictions_3.csv')

##Model 4 - 5 layers 20 nodes per layer

**Processing Time:** 156 s

**Training Set Accuracy:** 0.8982

**Test Set Accuracy:** 0.8992

In [28]:
%%time
feature_cols = [tf.feature_column.numeric_column("X", shape=[28 * 28])]
dnn_clf_4 = tf.estimator.DNNClassifier(hidden_units=[20,20,20,20,20], n_classes=10,
                                     feature_columns=feature_cols)

input_fn_4 = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, num_epochs=50, batch_size=50, shuffle=True)
dnn_clf_4.train(input_fn=input_fn_4)

CPU times: user 2min 36s, sys: 7.19 s, total: 2min 43s
Wall time: 1min 49s


In [29]:
# evaluate training accuracy
train_4_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_train}, y=y_train, shuffle=False)
eval_results_train_4 = dnn_clf_4.evaluate(input_fn=train_4_input_fn)
eval_results_train_4

{'accuracy': 0.8997,
 'average_loss': 0.36296853,
 'global_step': 60000,
 'loss': 0.36300045}

In [30]:
# evalute test accuracy
test_4_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={"X": X_test}, y=y_test, shuffle=False)
eval_results_test_4 = dnn_clf_4.evaluate(input_fn=test_4_input_fn)
eval_results_test_4

{'accuracy': 0.8996,
 'average_loss': 0.36112633,
 'global_step': 60000,
 'loss': 0.35851395}

In [31]:
#confusion matrix for Model 4 predictions
raw_predictions_4 = dnn_clf_4.predict(input_fn=test_4_input_fn)
predictions_4 = [p['class_ids'][0] for p in raw_predictions_4]
confusion_matrix_4 = tf.math.confusion_matrix(y_test, predictions_4)
confusion_matrix_4

<tf.Tensor: shape=(10, 10), dtype=int32, numpy=
array([[ 940,    0,    1,    2,    0,   10,   12,    7,    8,    0],
       [   0, 1100,    5,    6,    0,    3,    2,    0,   19,    0],
       [  15,    7,  901,   19,   11,    0,   27,    9,   39,    4],
       [   5,    7,   28,  883,    0,   37,    0,   18,   24,    8],
       [   0,    0,    5,    2,  891,    0,   20,    1,   14,   49],
       [  22,    6,    5,   33,    2,  752,    9,    5,   50,    8],
       [  36,    3,   10,    0,   12,    6,  884,    0,    7,    0],
       [   1,    4,   20,   27,    5,    1,    0,  924,   13,   33],
       [   9,    7,   19,   28,    7,   31,   15,    8,  837,   13],
       [   4,    4,    1,   12,   49,   11,    1,   25,   18,  884]],
      dtype=int32)>

In [32]:
print(predictions_4)

[7, 2, 1, 0, 4, 1, 4, 9, 8, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 6, 7, 2, 7, 1, 3, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 2, 7, 4, 2, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 3, 3, 9, 7, 4, 4, 4, 9, 8, 5, 4, 7, 6, 9, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 4, 9, 8, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 3, 3, 2, 3, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 9, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 7, 9, 3, 9, 0, 3, 5, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 7, 7, 9, 2, 2, 4, 1, 5, 8, 8, 7, 2, 0, 0, 2, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 0, 8, 5, 7, 7, 9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2, 6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 3, 6, 7, 1, 2, 4, 0, 2, 9, 4, 3, 3, 0, 0, 3, 1, 9, 6, 5, 2, 5, 3, 7, 9, 3, 5, 4, 2, 0, 7, 1, 1, 2, 1, 

In [33]:
kag_df4 = pd.DataFrame(predictions_4, columns= ['Label']).reset_index()
kag_df4['index'] = kag_df4.index + 1
kag_df4.rename(columns ={'index':'ImageId'}, inplace=True)
kag_df4.head()

,ImageId,Label
0,1,7
1,2,2
2,3,1
3,4,0
4,5,4


In [34]:
kag_df4.to_csv('Hilderbrand_DNN_Predictions_4.csv')

##Summary


Model 1 took the least amount of time (128 s) to process, but Model 2 had the highest accuracy score on the test data set (0.9142) only taking an additional 20 seconds with a processing time of 140 s.